# Data Driven Web Application Front-End

## Disease Prediction

In [1]:
from ipywidgets import Button, SelectMultiple, Dropdown, Accordion, Tab
from ipywidgets import VBox, HBox, HTML, FileUpload, widgets, Label, Layout
import pandas as pd
import ipyaggrid
import pickle
import requests
import json
import io

In [2]:
filename = 'data.pck'
loaded_data = pickle.load(open(filename, 'rb'))

In [3]:
# dropdowns
disease_desc = Dropdown(options=loaded_data['Diseases'], value='Common Cold',
                    description='Disease:', disabled=False)

# select multiple
symptoms = SelectMultiple(options=loaded_data['Symptoms'],
                            value=['chills'],
                            rows=10,
                            description='Symptoms',
                            disabled=False)

# HTML
details_disease = HTML(value= "", placeholder='disease details', description='')
predict_sym = HTML(value= "", placeholder='disease details', description='')

# Tab
tab = Tab(layout=Layout(height='500px'))

# FileUpload
uploaded_file = FileUpload(accept='', multiple=False)

# Grid options
grid_options = {'enableSorting': True, 'enableFilter': True, 'enableColResize': True, 'enableRangeSelection': True}

In [4]:
# buttons
def onClickSearchDisease(btn):
    data = disease_desc.value
    r = requests.get('http://127.0.0.1:8001/search',json={'data':data})
    msg = r.json()['msg']
    desc = r.json()['description']
    prec = r.json()['precaution']
    details_disease.value = "<h3><b>"+msg+"</b></h3>"+"<b>Description: </b>"+desc+"<br><b>Precaution: </b><br>"+prec

search_disease = Button(description='Search', disabled=False,
                        button_style='info',
                        tooltip='search description and precaution',
                        icon='search')
search_disease.on_click(onClickSearchDisease)
#----------------------------------------------------------------------------------------------------------------
def onClickPredictDisease(btn):
    data = dict.fromkeys(loaded_data['Symptoms'], 0.0)
    for value in symptoms.value:
        data[value] = 1.0
    
    r = requests.post('http://127.0.0.1:8001/model',json={'data':[list(data.values())]})
    
    predict_sym.value = "<br><b>Predicted Disease:</b> "+r.json()['msg'][0]
    
predict_disease = Button(description='Predict', disabled=False,
                        button_style='success',
                        tooltip='disease prediction by symptoms',
                        icon='question')
predict_disease.on_click(onClickPredictDisease)
#----------------------------------------------------------------------------------------------------------------
def onClickPredictDisease2(btn):
    if uploaded_file.value != {}:
        content = uploaded_file.value[list(uploaded_file.value.keys())[0]]['content']
        df = pd.read_csv(io.BytesIO(content))
        data = pd.DataFrame(columns=loaded_data['Symptoms'])
        
        for row in range(df.shape[0]):
            columns = df.loc[row].tolist()[1:]
            columns = [x.replace(" ","") for x in columns if pd.isnull(x) == False]
            new_row = dict.fromkeys(loaded_data['Symptoms'], 0.0)
            for col in columns:
                new_row[col] = 1.0
            data = data.append(new_row, ignore_index=True)
        
        r = requests.post('http://127.0.0.1:8001/model',json={'data':data.values.tolist()})
        value = r.json()['msg']
        
        df.insert(0, 'Disease', value, allow_duplicates = True)
        
        
        
        table = ipyaggrid.Grid(grid_data=df, grid_options=grid_options, quick_filter=True, 
                               show_toggle_edit=True, export_mode="buttons", export_csv=True, export_excel=True,
                               theme='ag-theme-balham', show_toggle_delete=True, columns_fit='auto', 
                               index=False, keep_multiindex=False)

        
        tab.children = [table]
        tab.set_title(0,"Predicted Table")
        
        

predict_disease2 = Button(description='Predict', disabled=False,
                        button_style='success',
                        tooltip='disease prediction by symptoms',
                        icon='question')
predict_disease2.on_click(onClickPredictDisease2)

In [5]:
child1 = VBox([HBox([disease_desc,search_disease]), details_disease])
child2 = VBox([symptoms,predict_disease,predict_sym])
child3 = VBox([uploaded_file, predict_disease2])
child4 = VBox([tab])

In [6]:
accordion = Accordion(children=[child1, child2, child3])
accordion.set_title(0, 'Disease Details')
accordion.set_title(1, 'Predict Disease by Symptom')
accordion.set_title(2, 'Predict Disease through upload file')


In [7]:
display(VBox([accordion, child4]))